# Exploratory Data Analysis

In [2]:
import requests
import json
from collections import Counter

In [3]:
url = "https://api.scryfall.com/bulk-data"
resp = requests.get(url)
print(resp)
j = resp.json()
print(j.keys())
for d in j["data"]:
    print(d)

<Response [200]>
dict_keys(['object', 'has_more', 'data'])
{'object': 'bulk_data', 'id': '27bf3214-1271-490b-bdfe-c0be6c23d02e', 'type': 'oracle_cards', 'updated_at': '2023-09-17T21:02:10.589+00:00', 'uri': 'https://api.scryfall.com/bulk-data/27bf3214-1271-490b-bdfe-c0be6c23d02e', 'name': 'Oracle Cards', 'description': 'A JSON file containing one Scryfall card object for each Oracle ID on Scryfall. The chosen sets for the cards are an attempt to return the most up-to-date recognizable version of the card.', 'size': 130989584, 'download_uri': 'https://data.scryfall.io/oracle-cards/oracle-cards-20230917210210.json', 'content_type': 'application/json', 'content_encoding': 'gzip'}
{'object': 'bulk_data', 'id': '6bbcf976-6369-4401-88fc-3a9e4984c305', 'type': 'unique_artwork', 'updated_at': '2023-09-17T21:02:43.204+00:00', 'uri': 'https://api.scryfall.com/bulk-data/6bbcf976-6369-4401-88fc-3a9e4984c305', 'name': 'Unique Artwork', 'description': 'A JSON file of Scryfall card objects that toget

In [4]:
download_uri = j["data"][3]["download_uri"]
print(download_uri)
resp = requests.get(download_uri)
print(resp)

https://data.scryfall.io/all-cards/all-cards-20230917211913.json
<Response [200]>


In [5]:
filename = download_uri.rsplit("/", 1)[-1]
with open(f"data/{filename}", "w", encoding="UTF-8") as f:
    json.dump(resp.json(), f)

In [6]:
with open(f"data/{filename}", "r", encoding="UTF-8") as f:
    cards = json.load(f)

In [7]:
def get_attribute(list_of_dicts: list, attribute_name: str) -> list:
    """Return a list containing the values of the attributes"""
    values = list()
    missing_count = 0
    # if not all([type(d) is dict for d in list_of_dicts]):
    #     print(type(list_of_dicts[0]))
    #     raise Exception
    for d in list_of_dicts:
        if attribute_name in d:
            values.append(d[attribute_name])
        else:
            missing_count += 1
    # print(f"{missing_count} cards were missing the attribute {attribute_name}.")
    return (values, missing_count)

In [8]:
def get_lengths(iterable: list) -> set:
    """Return a set containing all possible lengths for the values in iterable"""
    lengths = set()
    for e in iterable:
        lengths.add(len(e))
    return lengths

In [9]:
def get_types(iterable: list) -> set:
    """Return a set containing all possible data types for the values in iterable"""
    types = set()
    for e in iterable:
        types.add(type(e))
    return types

## Metadata

In [10]:
len(cards)

431784

In [11]:
attributes = set()
for card in cards:
    for key in card.keys():
        attributes.add(key)
attributes

{'all_parts',
 'arena_id',
 'artist',
 'artist_ids',
 'attraction_lights',
 'booster',
 'border_color',
 'card_back_id',
 'card_faces',
 'cardmarket_id',
 'cmc',
 'collector_number',
 'color_identity',
 'color_indicator',
 'colors',
 'content_warning',
 'digital',
 'edhrec_rank',
 'finishes',
 'flavor_name',
 'flavor_text',
 'foil',
 'frame',
 'frame_effects',
 'full_art',
 'games',
 'hand_modifier',
 'highres_image',
 'id',
 'illustration_id',
 'image_status',
 'image_uris',
 'keywords',
 'lang',
 'layout',
 'legalities',
 'life_modifier',
 'loyalty',
 'mana_cost',
 'mtgo_foil_id',
 'mtgo_id',
 'multiverse_ids',
 'name',
 'nonfoil',
 'object',
 'oracle_id',
 'oracle_text',
 'oversized',
 'penny_rank',
 'power',
 'preview',
 'prices',
 'printed_name',
 'printed_text',
 'printed_type_line',
 'prints_search_uri',
 'produced_mana',
 'promo',
 'promo_types',
 'purchase_uris',
 'rarity',
 'related_uris',
 'released_at',
 'reprint',
 'reserved',
 'rulings_uri',
 'scryfall_set_uri',
 'scryfal

In [12]:
print(len(attributes))

85


In [13]:
def get_metadata(list_of_objs: list, attributes: set):
    """Return a dictionary containing metadata about the list_of_objs passed to it"""
    metadata = dict()
    for attribute in attributes:
        data = dict()
        attribute_values, missing_count = get_attribute(list_of_objs, attribute)
        data["missing_count"] = missing_count
        types = [str(t) for t in get_types(attribute_values)]
        data["types"] = types
        if types == ["<class 'str'>"]:
            data["lengths"] = list(get_lengths(attribute_values))
        else:
            data["lengths"] = None
        try:
            distinct_count = len(set(attribute_values))
        except TypeError:
            distinct_count = None
        data["distinct_value_count"] = distinct_count
        metadata[attribute] = data
    return metadata

In [14]:
metadata = get_metadata(cards, attributes)
metadata

{'story_spotlight': {'missing_count': 0,
  'types': ["<class 'bool'>"],
  'lengths': None,
  'distinct_value_count': 2},
 'keywords': {'missing_count': 0,
  'types': ["<class 'list'>"],
  'lengths': None,
  'distinct_value_count': None},
 'cmc': {'missing_count': 18,
  'types': ["<class 'float'>"],
  'lengths': None,
  'distinct_value_count': 19},
 'set_uri': {'missing_count': 0,
  'types': ["<class 'str'>"],
  'lengths': [66],
  'distinct_value_count': 856},
 'nonfoil': {'missing_count': 0,
  'types': ["<class 'bool'>"],
  'lengths': None,
  'distinct_value_count': 2},
 'booster': {'missing_count': 0,
  'types': ["<class 'bool'>"],
  'lengths': None,
  'distinct_value_count': 2},
 'frame_effects': {'missing_count': 396587,
  'types': ["<class 'list'>"],
  'lengths': None,
  'distinct_value_count': None},
 'set_search_uri': {'missing_count': 0,
  'types': ["<class 'str'>"],
  'lengths': [72, 73, 74, 71],
  'distinct_value_count': 856},
 'printed_name': {'missing_count': 100930,
  'type

In [15]:
with open("data/metadata.json", "w") as f:
    json.dump(metadata, f, indent=4)

Take the time to view the generated metadata.json file. It dictates the data types for the database.

## released_at

In [16]:
released_ats = get_attribute(cards, "released_at")
released_ats[:5]

(['2006-10-06',
  '2009-10-02',
  '2017-08-25',
  '2015-05-22',
  '2017-09-29',
  '2002-05-27',
  '1994-04-01',
  '2016-09-30',
  '2005-10-07',
  '2021-09-24',
  '2021-06-18',
  '2021-11-19',
  '2012-10-05',
  '2005-10-07',
  '2001-10-01',
  '2022-11-18',
  '2022-04-29',
  '2023-09-08',
  '2023-06-23',
  '2019-10-04',
  '2015-01-17',
  '2019-07-12',
  '2007-10-12',
  '2018-07-13',
  '2020-07-03',
  '2020-11-20',
  '2020-07-17',
  '2017-04-28',
  '1997-10-14',
  '2022-09-09',
  '2015-07-17',
  '2021-06-21',
  '2011-01-10',
  '2021-03-19',
  '2000-06-05',
  '2022-04-29',
  '2014-05-02',
  '2021-02-05',
  '2013-09-27',
  '2023-08-04',
  '2011-09-30',
  '2011-07-15',
  '2015-11-13',
  '2021-11-19',
  '1995-06-03',
  '2018-08-09',
  '2020-09-25',
  '2017-09-29',
  '2018-08-09',
  '1997-03-24',
  '2008-01-01',
  '2012-07-13',
  '2009-02-06',
  '1995-04-01',
  '1998-06-24',
  '2018-01-19',
  '2006-10-06',
  '2013-05-03',
  '2023-04-21',
  '2011-05-13',
  '1993-12-10',
  '2019-10-04',
  '2017-

The released_at field in the data is saved as a string but will be converted to a data in the database.

## collector_number

In [17]:
collector_numbers, _ = get_attribute(cards, "collector_number")
collector_numbers[:5]

['157', '21', '73', '5', '78']

It seems like the collector_number is usually an integer yet they were saved as strings. Why?

In [18]:
distinct_collector_numbers = set(collector_numbers)
distinct_collector_numbers

{'1118',
 '88280',
 '28a',
 '72†',
 '1001',
 '134★',
 'ab0b',
 'gb42b',
 '61557',
 '97911',
 'rb55sb',
 '693',
 '49834',
 'shr351',
 '36847',
 '856',
 '103406',
 'J9',
 '46896',
 '646',
 '95459',
 '105610',
 '90162',
 '224★',
 '72227',
 '105766',
 '81868',
 '35074',
 'A-183',
 '60454',
 'pp251',
 '2020-5',
 'E41',
 '145†',
 '46889',
 '86040',
 'tvdl19',
 '12c',
 '65763',
 '676',
 'pm37a',
 '32011',
 '1177',
 '92754',
 '65b',
 '995',
 'A64',
 'A138',
 '52322',
 '86280',
 '1374',
 '357',
 'jk34',
 '12g',
 'js45',
 '77975',
 '79891',
 '538',
 '177s',
 'ab69',
 '26★',
 '37612',
 '32196',
 '55735',
 '36214',
 'A111',
 '241a',
 '211c',
 '104',
 '341',
 '65857',
 '90338',
 '64s',
 '32575',
 '35158',
 '95321',
 'A-127',
 'bk331',
 '86276',
 '99773',
 '86050',
 '79925',
 '90092',
 'bh16',
 '86066',
 '31965',
 '202s',
 '256as',
 '163',
 '91205',
 '43069',
 '102309',
 '93a',
 '79941',
 'H17',
 '60474',
 '62433',
 '94080',
 'tvdl18sb',
 'H25',
 'sg287sb',
 '53826',
 'F58',
 'shh0b',
 'pk68sb',
 '2

collector_number is not always an integer.

## life_modifier

In [19]:
life_modifers, _ = get_attribute(cards, "life_modifier")
print(len(life_modifers))
print(set(life_modifers))

119
{'+18', '+10', '-2', '+2', '+5', '+7', '+4', '+0', '+1', '-3', '+15', '+9', '+3', '-7', '-6', '-4', '-5', '+30', '+8', '-8', '+6', '+12', '-1'}


## cmc

In [20]:
cmcs, _ = get_attribute(cards, "cmc")
print(set(cmcs))

{0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 0.5, 1000000.0}


## mana_cost

In [21]:
mana_costs, _ = get_attribute(cards, "mana_cost")
for mana_cost in mana_costs:
    if len(mana_cost) == 46:   # from metadata.json
        print(mana_cost)

{X}{W} // {2}{R} // {2}{U} // {3}{B} // {1}{G}
{X}{W} // {2}{R} // {2}{U} // {3}{B} // {1}{G}


## frame

In [22]:
frames, _ = get_attribute(cards, "frame")
print(set(frames))

{'future', '1997', '2015', '1993', '2003'}


## all_parts

In [23]:
all_parts, _ = get_attribute(cards, "all_parts")
all_parts[:5]   # Currently a list of lists of dictionaries

[[{'object': 'related_card',
   'id': '4d8542f6-ee34-42c6-acd5-07b0c7cc2f63',
   'component': 'combo_piece',
   'name': 'Funeral Pyre',
   'type_line': 'Instant',
   'uri': 'https://api.scryfall.com/cards/4d8542f6-ee34-42c6-acd5-07b0c7cc2f63'},
  {'object': 'related_card',
   'id': '66210a3f-010b-4a9b-a08f-97d3ca962b0c',
   'component': 'combo_piece',
   'name': 'Haunted Dead',
   'type_line': 'Creature — Zombie',
   'uri': 'https://api.scryfall.com/cards/66210a3f-010b-4a9b-a08f-97d3ca962b0c'},
  {'object': 'related_card',
   'id': 'd333e35c-ca90-4aaa-950a-48b5623c31a6',
   'component': 'combo_piece',
   'name': 'Blessed Defiance',
   'type_line': 'Instant',
   'uri': 'https://api.scryfall.com/cards/d333e35c-ca90-4aaa-950a-48b5623c31a6'},
  {'object': 'related_card',
   'id': 'f0ad0796-0357-4e74-9d65-c7761a3f223c',
   'component': 'combo_piece',
   'name': "Slayer's Plate",
   'type_line': 'Artifact — Equipment',
   'uri': 'https://api.scryfall.com/cards/f0ad0796-0357-4e74-9d65-c7761a3

In [24]:
get_types(all_parts)

{list}

In [25]:
get_lengths(all_parts)

{2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 23,
 24,
 25,
 26,
 29,
 31,
 32,
 33,
 34,
 36,
 37,
 38,
 39,
 42,
 43,
 45,
 47,
 50,
 53,
 55,
 60,
 65,
 68,
 70,
 80,
 89,
 95,
 97,
 191,
 224}

In [26]:
parts = list()
for part_grouping in all_parts:
    for part in part_grouping:
        parts.append(part)
parts[:5]

[{'object': 'related_card',
  'id': '4d8542f6-ee34-42c6-acd5-07b0c7cc2f63',
  'component': 'combo_piece',
  'name': 'Funeral Pyre',
  'type_line': 'Instant',
  'uri': 'https://api.scryfall.com/cards/4d8542f6-ee34-42c6-acd5-07b0c7cc2f63'},
 {'object': 'related_card',
  'id': '66210a3f-010b-4a9b-a08f-97d3ca962b0c',
  'component': 'combo_piece',
  'name': 'Haunted Dead',
  'type_line': 'Creature — Zombie',
  'uri': 'https://api.scryfall.com/cards/66210a3f-010b-4a9b-a08f-97d3ca962b0c'},
 {'object': 'related_card',
  'id': 'd333e35c-ca90-4aaa-950a-48b5623c31a6',
  'component': 'combo_piece',
  'name': 'Blessed Defiance',
  'type_line': 'Instant',
  'uri': 'https://api.scryfall.com/cards/d333e35c-ca90-4aaa-950a-48b5623c31a6'},
 {'object': 'related_card',
  'id': 'f0ad0796-0357-4e74-9d65-c7761a3f223c',
  'component': 'combo_piece',
  'name': "Slayer's Plate",
  'type_line': 'Artifact — Equipment',
  'uri': 'https://api.scryfall.com/cards/f0ad0796-0357-4e74-9d65-c7761a3f223c'},
 {'object': 're

In [27]:
# Get attributes of the all_parts object
attributes = set()
for part_grouping in all_parts:
    # print(type(part_grouping))
    # print(part_grouping)
    for part in part_grouping:
        for key in part.keys():
            attributes.add(key)
attributes

{'component', 'id', 'name', 'object', 'type_line', 'uri'}

In [28]:
parts_metadata = get_metadata(parts, attributes)

In [29]:
with open("data/all_parts_metadata.json", "w") as f:
    json.dump(parts_metadata, f, indent=4)

## color_indicator

In [35]:
color_indicators, _ = get_attribute(cards, "color_indicator")
color_indicators[:10]

[['R'], ['G'], ['W'], ['U'], ['W'], ['G'], ['B', 'R'], ['G'], ['G'], ['G']]

In [33]:
get_types(color_indicators)

{list}

## promo_types

In [36]:
promo_types, _ = get_attribute(cards, "promo_types")

In [37]:
unique_promo_types = set()
for promo_types_group in promo_types:
    for promo_type in promo_types_group:
        unique_promo_types.add(promo_type)
unique_promo_types

{'alchemy',
 'ampersand',
 'arenaleague',
 'boosterfun',
 'boxtopper',
 'brawldeck',
 'bringafriend',
 'bundle',
 'buyabox',
 'commanderparty',
 'concept',
 'confettifoil',
 'convention',
 'datestamped',
 'doublerainbow',
 'draculaseries',
 'draftweekend',
 'duels',
 'event',
 'fnm',
 'galaxyfoil',
 'gameday',
 'giftbox',
 'gilded',
 'glossy',
 'godzillaseries',
 'halofoil',
 'instore',
 'intropack',
 'jpwalker',
 'judgegift',
 'league',
 'mediainsert',
 'moonlitland',
 'neonink',
 'oilslick',
 'openhouse',
 'planeswalkerdeck',
 'plastic',
 'playerrewards',
 'playpromo',
 'premiereshop',
 'prerelease',
 'promopack',
 'rebalanced',
 'release',
 'schinesealtart',
 'serialized',
 'setextension',
 'setpromo',
 'stamped',
 'starterdeck',
 'stepandcompleat',
 'storechampionship',
 'surgefoil',
 'textured',
 'themepack',
 'thick',
 'tourney',
 'wizardsplaynetwork'}

In [65]:
len(unique_promo_types)

60

In [38]:
get_lengths(unique_promo_types)

{3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18}

## multiverse_ids

In [39]:
multiverse_ids, _ = get_attribute(cards, "multiverse_ids")

In [40]:
get_types(multiverse_ids)

{list}

In [42]:
flat_multiverse_ids = [multiverse_id for multiverse_ids_group in multiverse_ids for multiverse_id in multiverse_ids_group]
get_types(flat_multiverse_ids)

{int}

## frame_effects

In [45]:
frame_effects, _ = get_attribute(cards, "frame_effects")
frame_effects[:5]

[['legendary'], ['legendary'], ['nyxtouched'], ['nyxtouched'], ['legendary']]

In [46]:
flat_frame_effects = [frame_effect for frame_effects_group in frame_effects for frame_effect in frame_effects_group]

In [47]:
get_types(flat_frame_effects)

{str}

In [48]:
get_lengths(flat_frame_effects)

{4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 22}

In [49]:
set(flat_frame_effects)

{'colorshifted',
 'companion',
 'compasslanddfc',
 'convertdfc',
 'devoid',
 'draft',
 'etched',
 'extendedart',
 'fandfc',
 'fullart',
 'gravestone',
 'inverted',
 'legendary',
 'lesson',
 'miracle',
 'mooneldrazidfc',
 'nyxtouched',
 'originpwdfc',
 'shatteredglass',
 'showcase',
 'snow',
 'sunmoondfc',
 'textless',
 'tombstone',
 'upsidedowndfc',
 'waxingandwaningmoondfc'}

In [66]:
len(set(flat_frame_effects))

26

## attraction_lights

In [53]:
attraction_lights, _ = get_attribute(cards, "attraction_lights")

In [54]:
get_types(attraction_lights)

{list}

In [55]:
get_lengths(attraction_lights)

{2, 3, 4}

## finishes

In [56]:
finishes, _ = get_attribute(cards, "finishes")

In [57]:
get_types(finishes)

{list}

In [58]:
flat_finishes = [finish for finish_group in finishes for finish in finish_group]
set(flat_finishes)

{'etched', 'foil', 'nonfoil'}

## keywords

In [59]:
keywords, _ = get_attribute(cards, "keywords")

In [60]:
get_types(keywords)

{list}

In [62]:
flat_keywords = [keyword for keyword_group in keywords for keyword in keyword_group]
unique_keywords = set(flat_keywords)
unique_keywords

{'A Thousand Souls Die Every Day',
 'Aberrant Tinkering',
 'Adamant',
 'Adapt',
 'Addendum',
 'Advanced Species',
 'Aegis of the Emperor',
 'Affinity',
 'Afflict',
 'Afterlife',
 'Aftermath',
 'Aim for the Cursed Amulet',
 'Aim for the Wyvern',
 'Alliance',
 'Allure of Slaanesh',
 'Amass',
 'Amplify',
 'Animate Chains',
 'Annihilator',
 'Arcane Life-support',
 'Architect of Deception',
 'Armour of Shrieking Souls',
 'Ascend',
 'Assemble',
 'Assist',
 'Atomic Transmutation',
 'Augment',
 'Aura Swap',
 'Avoidance',
 'Awaken',
 'Backup',
 'Banding',
 'Bargain',
 'Basic landcycling',
 'Battalion',
 'Battle Cannon',
 'Battle Cry',
 'Benediction of the Omnissiah',
 'Berzerker',
 'Bestow',
 "Bigby's Hand",
 'Bio-Plasmic Scream',
 'Bio-plasmic Barrage',
 'Blitz',
 'Blood Chalice',
 'Blood Drain',
 'Bloodrush',
 'Bloodthirst',
 'Boast',
 'Body Thief',
 'Bolster',
 'Bribe the Guards',
 'Bring it Down!',
 'Brood Telepathy',
 'Bushido',
 'Buy Information',
 'Buyback',
 "Calim's Breath",
 'Call for

In [63]:
len(unique_keywords)

490

In [64]:
get_lengths(unique_keywords)

{4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 27,
 28,
 30}